In [5]:
import sys

sys.path.append("../")

import numpy as np
import pandas_ta as ta
import joblib

from terminal.Extensions.MT5Ext import MT5Ext
from datetime import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [6]:
# Modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [7]:
def feature_target(file):
    # Carregar os dados
    ticks = np.load(file, allow_pickle=True)["ticks"]
    data = MT5Ext.create_ohlc_from_ticks(ticks, "5s")

    del ticks

    # Cálculo de indicadores
    data.ta.macd(append=True)
    data.dropna(inplace=True)

    # Pré-processamento de dados
    data["change"] = data["close"].pct_change()
    data["target"] = np.where(data["change"] > 0, 1, 0)
    data["target"] = data["target"].shift(-1)
    data.dropna(inplace=True)

    # Criar características e rótulos
    feature_columns = [
        "open",
        "close",
        "MACD_12_26_9",
        "MACDh_12_26_9",
        "MACDs_12_26_9",
    ]
    features = data[feature_columns]
    target = data["target"]

    del data

    return features, target


def test(file):
    features, target = feature_target(file)

    # Avaliar o modelo
    y_pred = model.predict(features)
    accuracy = accuracy_score(target, y_pred)
    print(f"accuracy: {accuracy:.2f}")


def train(file):
    features, target = feature_target(file)

    model.fit(features, target)

In [8]:
for file in [
    "./data/ticks_WIN$N_2024_1.npz",
    "./data/ticks_WIN$N_2024_2.npz",
    "./data/ticks_WIN$N_2024_3.npz",
    "./data/ticks_WIN$N_2024_4.npz",
    "./data/ticks_WIN$N_2024_5.npz",
    "./data/ticks_WIN$N_2024_6.npz",
]:
    print(f"{datetime.now()} - train '{file}'")
    train(file)

2024-07-14 14:48:57.459576 - train './data/ticks_WIN$N_2024_1.npz'
2024-07-14 14:51:18.470013 - train './data/ticks_WIN$N_2024_2.npz'
2024-07-14 14:52:55.162443 - train './data/ticks_WIN$N_2024_3.npz'
2024-07-14 14:54:32.275973 - train './data/ticks_WIN$N_2024_4.npz'
2024-07-14 14:56:26.149626 - train './data/ticks_WIN$N_2024_5.npz'
2024-07-14 14:58:10.172024 - train './data/ticks_WIN$N_2024_6.npz'


In [9]:
test("./data/ticks_WIN$N_2024_7.npz")

accuracy: 0.57


In [10]:
# save
joblib.dump(model, "../models/WIN$N-2024_from_01_to_06-RandomForestClassifier-02.joblib")

['../models/WIN$N-2024_from_01_to_06-RandomForestClassifier-02.joblib']